In [ ]:
import numpy as np
import scipy.linalg as la

alpha = 1.0
T = 0.1

def initial_condition(x):
    return np.sin(np.pi * x)

def exact_solution(x, t):
    return np.exp(-(np.pi**2) * t) * np.sin(np.pi * x)

def laplacian_matrix_1d(N):
    # Tridiagonal T for interior points: -2 on diag, +1 on off-diags
    T = np.zeros((N, N))
    np.fill_diagonal(T, -2.0)
    np.fill_diagonal(T[1:], 1.0)
    np.fill_diagonal(T[:,1:], 1.0)
    return T

def amplification_matrix_ftcs(N, r):
    # G = I + r * T, where T is Laplacian stencil
    I = np.eye(N)
    T = laplacian_matrix_1d(N)
    return I + r * T

def run_ftcs(N, T):
    L = 1.0
    dx = L / (N + 1)
    dt = 0.4 * dx**2 / alpha   # r = 0.4 stable
    r = alpha * dt / dx**2     # should be 0.4

    x = np.linspace(dx, L - dx, N)
    u = initial_condition(x).copy()

    steps = int(np.round(T / dt))
    # Enforce final time exactly by adjusting last dt if needed
    if steps == 0:
        steps = 1
    t = 0.0

    # Precompute for FTCS: u^{n+1} = u^n + r*(u_{i-1}^n - 2u_i^n + u_{i+1}^n)
    for n in range(steps):
        un = u.copy()
        # boundaries are zero; use ghost values u_0 = u_{N+1} = 0
        u[1:-1] = un[1:-1] + r * (un[:-2] - 2*un[1:-1] + un[2:])
        u[0] = un[0] + r * (0.0 - 2*un[0] + un[1])
        u[-1] = un[-1] + r * (un[-2] - 2*un[-1] + 0.0)
        t += dt

    # Amplification matrix and its spectral radius
    G = amplification_matrix_ftcs(N, r)
    amplification_spectral_radius = np.max(np.abs(la.eigvals(G))).real

    # Error at final time T (approx)
    u_exact = exact_solution(x, t)
    ftcs_error = float(np.linalg.norm(u - u_exact) * np.sqrt(dx))  # L2 over domain (discrete)

    return u, ftcs_error, amplification_spectral_radius, dx, dt

def run_cn(N, T):
    L = 1.0
    dx = L / (N + 1)
    dt = 0.4 * dx**2 / alpha
    r = alpha * dt / dx**2

    x = np.linspace(dx, L - dx, N)
    u = initial_condition(x).copy()

    Tmat = laplacian_matrix_1d(N)
    I = np.eye(N)

    A = I - 0.5 * r * Tmat
    B = I + 0.5 * r * Tmat

    # Condition number (2-norm)
    A_CN_condition_number = float(np.linalg.cond(A))

    steps = int(np.round(T / dt))
    if steps == 0:
        steps = 1
    t = 0.0

    for n in range(steps):
        rhs = B @ u
        # Dirichlet BCs are already embedded (interior-only system)
        u = la.solve(A, rhs, assume_a='gen')
        t += dt

    u_exact = exact_solution(x, t)
    cn_error = float(np.linalg.norm(u - u_exact) * np.sqrt(dx))

    return u, cn_error, A_CN_condition_number, dx, dt

# Convergence analysis
convergence_data = {}
observed_orders = {}

for N in [20, 40, 80]:
    ftcs_solution, ftcs_error, amplification_spectral_radius, dx, dt = run_ftcs(N, T)
    cn_solution, cn_error, A_CN_condition_number, dx, dt = run_cn(N, T)
    convergence_data[N] = (ftcs_error, cn_error)

# Observed orders for CN
def order(e1, e2):
    return np.log2(e1 / e2)

cn_e20 = convergence_data[20][1]
cn_e40 = convergence_data[40][1]
cn_e80 = convergence_data[80][1]
observed_orders[(20,40)] = float(order(cn_e20, cn_e40))
observed_orders[(40,80)] = float(order(cn_e40, cn_e80))

# Stability for last run (N=80)
N = 80
ftcs_solution, ftcs_error, amplification_spectral_radius, dx, dt = run_ftcs(N, T)
cn_solution, cn_error, A_CN_condition_number, dx, dt = run_cn(N, T)

r_last = alpha * dt / dx**2
stability_ok = bool(r_last <= 0.5)

stability_report = {
    "r": float(r_last),
    "ftcs_stable": stability_ok,
    "amplification_spectral_radius": float(amplification_spectral_radius),
    "cn_unconditionally_stable": True,
    "A_CN_condition_number": float(A_CN_condition_number),
}

best_method = "crank_nicolson" if cn_error < ftcs_error else "ftcs"
